In [210]:
""" 
Import necessary libraries for performing mathematical and visulaization 
"""

import pandas as pd
import datetime as dt
import numpy as np
import json
import requests
import urllib.request
import datetime
from pathlib import Path
from dotenv import load_dotenv
import os
from blackscholesmodel import euro_vanilla, implied_vol, delta, vega, get_d1_d2

pd.set_option("mode.chained_assignment", None)

option_path = Path("/option_data")

load_dotenv('example.env')
iex_token = os.getenv("IEX_token")
# print(type(iex_token))


def find_atm_strike(strikes, cur_price): 
    """ This function returns closest strike price from the current stock price. 
    It take list of strike prices and current price as input.
    """
    
    min_diff = None
    atm_strike = None
    
    for strike in strikes:
        if min_diff == None: 
            min_diff = abs(strike - cur_price) 
        elif abs(strike - cur_price)  < min_diff: 
            min_diff = abs(strike - cur_price)
            atm_strike = strike
            
    return atm_strike


In [350]:
"""
Retrieve End of the Day Option Prices from IEX Cloud for tickes in scope, one at a time. 
Load the Options data into pandas dataframe. 

"""

tickers = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'TWTR', 'JPM', 'DAL', 'PFE', 'MRNA','XOM', 'CVX','AMZN','ZM','FB', 'GILD', 'SPY'] 

locally_loaded = [] 

for ticker in tickers: 
    fname = '.'.join([ticker, 'csv'])
    file_path = Path(". option_path/" + fname)

    df_flag = False
    
    try: 
        option_df = pd.read_csv('option_data/' + fname, )
        locally_loaded.append(ticker)
        df_flag = True
    except Exception as e: 
        print(e)
    
    if not df_flag:
        try:
            # Retrive available option matuirties for the ticker from IEX 
            
            url = 'https://cloud.iexapis.com/stable/stock/' + ticker + '/options/?token=' + iex_token
            option_maturities  = requests.get(url).json()
            maturities = ['20210319', '20210416']
        except Exception as e:
            print(e)

        try: 
            option_dfs = []
            for date in maturities: 
                
                # retrieve the Option Chain for each maturity from IEX 
                
                url = 'https://cloud.iexapis.com/stable/stock/' + ticker + '/options/' + date + '?token=' + iex_token
                option_data = requests.get(url).json()
                option_df = pd.DataFrame(option_data)
                option_df = option_df.loc[option_df['exerciseStyle'] == 'A']
                option_dfs.append(option_df)
        except Exception as e:
            print(e)
        
        # Combine options data for all maturities into single dataframe & export to local warehouse folder
        final_option_df = pd.concat(option_dfs, axis=0)
        final_option_df.to_csv("option_data/" + fname)
        
# option_df.head(5)
print(f'Loaded from local folder: {locally_loaded}')


Loaded from local folder: ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'TWTR', 'JPM', 'DAL', 'PFE', 'MRNA', 'XOM', 'CVX', 'AMZN', 'ZM', 'FB', 'GILD', 'SPY']


In [351]:
# Retrieve close price, Market Cap, 52 Week High/Low and other details from IEX Quotes

symbols = ','.join(tickers)
quote_url = 'https://cloud.iexapis.com/stable/stock/market/batch?symbols=' + symbols + '&types=quote&token=' + iex_token
cols = ['close', 'marketCap', 'week52High', 'week52Low']

hist_data = {}

try: 
    quotes = requests.get(quote_url).json()
    for k, v in quotes.items(): 
        
        for col in cols: 
            if k not in hist_data: 
                hist_data[k] = {}
            
            if col == 'close':
                if v['quote'][col] == None:
                    hist_data[k][col] = v['quote']['previousClose']
                else: 
                    hist_data[k][col] = v['quote'][col]
            else: 
                hist_data[k][col] = v['quote'][col]
                
except Exception as e:
    print(e)



In [354]:
print(hist_data)

{'AAPL': {'close': 129.71, 'marketCap': 2177583932160, 'week52High': 144.87, 'week52Low': 52.74}, 'MSFT': {'close': 243.79, 'marketCap': 1818277377108, 'week52High': 245.56, 'week52Low': 131.19}, 'GOOG': {'close': 2117.2, 'marketCap': 1423800621670, 'week52High': 2152.68, 'week52Low': 1013.54}, 'TSLA': {'close': 787.38, 'marketCap': 750293487739, 'week52High': 900.4, 'week52Low': 70.1}, 'TWTR': {'close': 72.26, 'marketCap': 57658535733, 'week52High': 74.96, 'week52Low': 20}, 'JPM': {'close': 145.59, 'marketCap': 452322852519, 'week52High': 148.55, 'week52Low': 74.16}, 'DAL': {'close': 44.38, 'marketCap': 29297313390, 'week52High': 58.02, 'week52Low': 17.51}, 'PFE': {'close': 34.56, 'marketCap': 191570138785, 'week52High': 42.62, 'week52Low': 26.77}, 'MRNA': {'close': 169.57, 'marketCap': 69059327525, 'week52High': 189.26, 'week52Low': 17.91}, 'XOM': {'close': 52.02, 'marketCap': 221496044062, 'week52High': 54.97, 'week52Low': 27.81}, 'CVX': {'close': 95, 'marketCap': 179094991898, 'wee

In [358]:
## IEX: Get historical closing prices, market cap, peRatio, 52 wk hi/low for16 tickers and place into df
locally_loaded = []
for ticker in tickers:
    fname = '.'.join([ticker, 'csv'])
    
    file_path = Path('/option_data/hist_price/' + fname)
    df_flag = False
    
    try:
        stock_df = pd.read_csv("option_data/hist_price/" + fname)
        locally_loaded.append(ticker)
        df_flag = True
    except Exception as e:
        print(e)
                     
    if not df_flag:
        print(ticker)
        try:
            url = 'https://cloud.iexapis.com/stable/stock/' + ticker + '/chart/1y?token=' + iex_token
            stock_chart = requests.get(url).json()
            stock_df = pd.DataFrame(stock_chart)
            stock_df.to_csv("option_data/hist_price/" + fname)
        except Exception as e:
            print(e)
                     
    # Calculate daily returns
    daily_returns = stock_df['close'].pct_change()
    # compute StDev
    rolling_std30 = daily_returns.rolling(window=30).std()
    # daily_std = daily_returns.std()
    annualized_std = (rolling_std30 * np.sqrt(252)).mean()
    
    # assign Hist Vol to existing hist_data dictionary, not new dictionary
    hist_data[ticker]['histVol'] = annualized_std

# print(hist_data)
print(f'Loaded from local folder: {locally_loaded}')

Loaded from local folder: ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'TWTR', 'JPM', 'DAL', 'PFE', 'MRNA', 'XOM', 'CVX', 'AMZN', 'ZM', 'FB', 'GILD', 'SPY']


In [359]:
## Retrive Treasury Rates from IEX Cloud and add to hist_data  dictionary
tenors = ['DGS3MO', 'DGS1MO']
tenor_data = {}
for tenor in tenors:
    url = 'https://cloud.iexapis.com/stable/time-series/treasury/' + tenor + '?token=' + iex_token
#     print(url)
    treasury_maturities  = requests.get(url).json()
#     print(type(treasury_maturities))
    for t in treasury_maturities:
        tenor_data[t['key']] = t['value']
hist_data['tenor_rate'] = tenor_data
tenor_data

{'DGS3MO': 0.04, 'DGS1MO': 0.03}

In [197]:
''' 
    Load Call & Put option data for different maurities into seperate Dataframe and store in Dictionary for later use.

'''
option_data= {}
tickers = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'TWTR', 'JPM', 'DAL', 'PFE', 'MRNA','XOM', 'CVX','AMZN','ZM','FB', 'GILD', 'SPY']
for ticker in tickers: 
    print(f'Processing {ticker} ...')
    option_data[ticker] = {}
    
    fname = '.'.join([ticker, 'csv'])
    cur_close = hist_data[ticker]['close']
#     print(cur_close)
    
    try: 
        option_df = pd.read_csv('option_data/' + fname)
                
    except Exception as e:
        print(e)
        continue
           
    call_option_df = option_df.loc[option_df['side'] == 'call'].sort_values(by='strikePrice', ascending=True)
    put_option_df = option_df.loc[option_df['side'] == 'put'].sort_values(by='strikePrice', ascending=True)
    cols = ['symbol', 'expirationDate', 'side','strikePrice', 'close', 'ask', 'bid', 'volume', 'openInterest']

    call_option_df = call_option_df[cols]
    put_option_df = put_option_df[cols]
    maturities = [20210319, 20210416]
    
    for expiry in maturities:
        option_data[ticker][expiry] = {}
        call_option_exp_df = call_option_df.loc[call_option_df['expirationDate'] == expiry]
        put_option_exp_df = put_option_df.loc[put_option_df['expirationDate'] == expiry]
        
        if put_option_exp_df.empty:
            continue
        
        put_strikes = put_option_exp_df['strikePrice'].tolist()
        put_atm_strike = find_atm_strike(put_strikes, cur_close)

        call_strikes = call_option_exp_df['strikePrice'].tolist()
        call_atm_strike = find_atm_strike(call_strikes, cur_close)

        put_option_exp_df = put_option_exp_df.reset_index()
        put_option_exp_df.drop('index', axis=1,  inplace=True)
        call_option_exp_df = call_option_exp_df.reset_index()
        call_option_exp_df.drop('index', axis=1,  inplace=True)

        put_atm_strike_index = put_option_exp_df.loc[put_option_exp_df['strikePrice']==put_atm_strike].index[0]
        call_atm_strike_index = call_option_exp_df.loc[call_option_exp_df['strikePrice']==call_atm_strike].index[0]

        put_lower_index = put_atm_strike_index - min(10, put_atm_strike_index)
        put_upper_index = put_atm_strike_index + min(11, put_option_exp_df.shape[0] - put_atm_strike_index)

        call_lower_index = call_atm_strike_index - min(10, call_atm_strike_index)
        call_upper_index = call_atm_strike_index + min(11, call_option_exp_df.shape[0] - call_atm_strike_index)

        final_put_df = put_option_exp_df.iloc[put_lower_index:put_upper_index, ]
        final_put_df = final_put_df.reset_index()
        final_put_df.drop('index', axis=1,  inplace=True)
        final_put_df['mid_price'] = (final_put_df['ask'] +  final_put_df['bid'])/2
        option_data[ticker][expiry]['put'] = final_put_df

        final_call_df = call_option_exp_df.iloc[call_lower_index:call_upper_index, ]
        final_call_df = final_call_df.reset_index()
        final_call_df.drop('index', axis=1,  inplace=True)
        final_call_df['mid_price'] = (final_call_df['ask'] +  final_call_df['bid'])/2
        option_data[ticker][expiry]['call'] = final_call_df

# print(option_data['AAPL'][20210319]['call'].head(5))
# option_data['AAPL'][20210416]['call'].tail(10)

Processing AAPL ...
Processing MSFT ...
Processing GOOG ...
Processing TSLA ...
Processing TWTR ...
Processing JPM ...
Processing DAL ...
Processing PFE ...
Processing MRNA ...
Processing XOM ...
Processing CVX ...
Processing AMZN ...
Processing ZM ...
Processing FB ...
Processing GILD ...
Processing SPY ...


In [209]:
# Calculate Implied Volatity & Options Greeks for option chain one ticker at a time and update relevant ticker dataframe in the Option Data dictionary. 

option_data_greeks= {}

r = 0.001
t = 30/365
hist_vol = 0.20

maturities = [20210319, 20210416]
# symbols = ['SPY']
for ticker in tickers: 
    s = hist_data[ticker]['close']
    print(f'{ticker}, stock price: {s}')
    
    for expiry in maturities:
        
        if not option_data[ticker][expiry]: 
            continue
        
        for side in ['call', 'put']:
#             print(option_data[ticker][expiry][side].head())
            option_data[ticker][expiry][side]['implied_vol'] = 1111111
            option_data[ticker][expiry][side]['delta'] = 1111111
            option_data[ticker][expiry][side]['vega'] = 1111111
            
            df2 = option_data[ticker][expiry][side].set_index('strikePrice')
            df3 = pd.Series(option_data[ticker][expiry][side].set_index('strikePrice')['mid_price'])
            callput = side
            strikes = list(df3.index)
            
            for strike in strikes:
                
                # Compute Implied volatility for each option strike using Black Scholes Model & Newton's method for root solving 
                
                mid_price = df3[strike]
                imp_vol = implied_vol(callput, s, strike, r, mid_price, t, hist_vol) 
                if imp_vol == 9999999:
                    print(f'Expiry: {expiry}, strike: {strike} : {side} option_price: {mid_price:.3f} vol: {imp_vol}')
                
                if imp_vol != 9999999:
                    # if the implied volatility is computed successfullt then calcutae Delta & Gama and add in to Option Dataframe
            
                    d1_option = get_d1_d2(s, strike, t, r, imp_vol)[0]
                    delta_option = delta(side, d1_option)
                    vega_option = vega(s, d1_option, t)
                    option_data[ticker][expiry][side]['implied_vol'].loc[option_data[ticker][expiry][side]['strikePrice'] == strike] = round(imp_vol * 100, 3)
                    option_data[ticker][expiry][side]['delta'].loc[option_data[ticker][expiry][side]['strikePrice'] == strike] = delta_option
                    option_data[ticker][expiry][side]['vega'].loc[option_data[ticker][expiry][side]['strikePrice'] == strike] = vega_option
            
            option_data[ticker][expiry][side] = option_data[ticker][expiry][side].loc[option_data[ticker][expiry][side]['implied_vol'] !=1111111]
                    
# option_data['AAPL'][20210319]['put'].tail(20)
# option_data['AAPL'][20210319]['call'].tail(20)

AAPL, stock price: 133.19
Does not Converge
Expiry: 20210319, strike: 107.5 : call option_price: 25.725 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 110.0 : call option_price: 23.500 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 115.0 : call option_price: 18.050 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 165.0 : call option_price: 0.225 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 170.0 : call option_price: 0.190 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 107.5 : put option_price: 0.355 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 110.0 : put option_price: 0.410 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 165.0 : put option_price: 32.100 vol: 9999999
Does not Converge
Expiry: 20210319, strike: 175.0 : put option_price: 42.050 vol: 9999999
Does not Converge
Expiry: 20210416, strike: 165.0 : put option_price: 32.575 vol: 9999999
MSFT, stock price: 243.14
Does not Converge
Expiry: 20210319, strike: 195

In [246]:
option_data['MSFT'][20210416]['put']
option_data['AMZN'][20210319]['put']

,symbol,expirationDate,side,strikePrice,close,ask,bid,volume,openInterest,mid_price,implied_vol,delta,vega
0,AMZN,20210319,put,3185,72.80,73.60,71.35,19,245,72.475,29.671,-0.363415,397.405291
1,AMZN,20210319,put,3190,74.52,75.45,73.20,19,350,74.325,29.630,-0.370228,394.965452
2,AMZN,20210319,put,3195,77.55,77.35,75.10,22,315,76.225,29.593,-0.377104,392.662244
3,AMZN,20210319,put,3200,79.00,79.35,77.05,133,5827,78.200,29.567,-0.384056,390.489960
4,AMZN,20210319,put,3210,83.55,83.30,81.35,19,273,82.325,29.535,-0.398118,386.555297
5,AMZN,20210319,put,3220,86.85,87.45,85.15,26,405,86.300,29.424,-0.412153,383.208674
6,AMZN,20210319,put,3230,91.00,91.80,89.40,16,172,90.600,29.362,-0.426434,380.365831
7,AMZN,20210319,put,3240,94.93,96.25,93.75,21,266,95.000,29.289,-0.440816,378.048023
8,AMZN,20210319,put,3250,98.70,100.90,98.50,76,648,99.700,29.259,-0.455320,376.242327
9,AMZN,20210319,put,3260,104.10,105.70,103.05,24,162,104.375,29.182,-0.469875,374.949767


In [323]:
# Concatenate put & call dfs of each ticker into final_option_data dictionary 

# create new dictionary option_data_conc
joint_option_data = {}
final_option_data = {}

symbols = ['JPM', 'SPY']
# tickers = ['AAPL', 'SPY', 'XOM', 'FB', 'TSLA', 'AMZN', 'CVX', 'GOOG', 'MRNA', 'MSFT', 'TWTR', 'JPM']

for ticker in tickers:
    joint_option_data[ticker] = {}
    
    option_df_list = []
    
    # set index to strikeprice for each put/call df
    
    for expiry in maturities:
        
        if not option_data[ticker][expiry]: 
            continue
            
        df1 = option_data[ticker][expiry]['call'].set_index('strikePrice')
        df1.drop(columns=['symbol', 'expirationDate', 'mid_price', 'close'], inplace=True)
        df1 = df1[['openInterest', 'volume', 'vega', 'implied_vol', 'delta', 'bid', 'ask', 'side']]
        df1.rename(columns={'openInterest': 'op_Int'}, inplace=True)
        df2 = option_data[ticker][expiry]['put'].set_index('strikePrice')
        df2.drop(columns=['symbol', 'expirationDate', 'mid_price', 'close'], inplace=True)
        df2 = df2[['side', 'ask', 'bid', 'delta', 'implied_vol', 'vega',  'volume', 'openInterest']]
        df2.rename(columns={'openInterest': 'op_Int'}, inplace=True)
        
        
        joint_option_data[ticker][expiry] = pd.concat([df1, df2], axis = 1, join = 'outer').reset_index()
        joint_option_data[ticker][expiry]['symbol'] = ticker
        joint_option_data[ticker][expiry]['exp_date'] = int(expiry)
        
        
    for expiry in maturities:
        
        if not option_data[ticker][expiry]: 
            continue
            
        option_df_list.append(joint_option_data[ticker][expiry])
        
    
    final_option_data[ticker]= pd.concat(option_df_list, axis = 0, join = 'inner')
    
    fname = ticker + '_option_chain.csv'
    
    final_option_data[ticker].to_csv("option_data/" + fname )

# print(joint_option_data['SPY'][20210319])
# print(final_option_data[ticker].tail(2))


C:\Users\KetanHina\study\cu-nyc-fin-pt-12-2020-u-c\03-Projects\Project-01


In [347]:

for ticker in final_option_data: 
    final_option_data[ticker]['Underlying_price'] = hist_data[ticker]['close']
    final_option_data[ticker]['hist_vol'] = round(hist_data[ticker]['histVol']*100, 3)

# hist_data['SPY']

In [348]:
# print(final_option_data.keys())

del final_option_dataframe

# final_option_dataframe = pd.DataFrame()
# print(final_option_dataframe.empty)
final_df_list = [] 

for k in final_option_data:
    final_df_list.append(final_option_data[k])
    
final_option_dataframe= pd.concat(final_df_list, axis = 0, join = 'inner')
final_option_dataframe = final_option_dataframe.reset_index()
final_option_dataframe.drop(columns=['index'], inplace=True)
    
    
# print(final_option_dataframe.shape)
# print(final_option_dataframe.columns)
# cols = ['symbol', 'exp_date','strikePrice', 'side', 'ask', 'bid', 'volume', 'openInterest', 'implied_vol', 'delta', 'vega', 'side', 'ask', 'bid', 'volume','openInterest',
#         'implied_vol', 'delta', 'vega']

final_option_dataframe.to_csv("option_data/Final_option_dataframe.csv" )
final_option_dataframe.head(20)
final_option_dataframe.tail(20)


,strikePrice,op_Int,volume,vega,implied_vol,delta,bid,ask,side,side,...,bid,delta,implied_vol,vega,volume,op_Int,symbol,exp_date,Underlying_price,hist_vol
426,383.0,3944.0,639.0,47.649730,27.291,0.635633,17.23,17.43,call,put,...,9.05,-0.372938,29.853,47.287830,326.0,2453.0,SPY,20210416,392.3,25.177
427,384.0,5904.0,31.0,47.167975,26.985,0.624057,16.50,16.70,call,put,...,9.33,-0.383669,29.534,46.875951,45.0,4085.0,SPY,20210416,392.3,25.177
428,385.0,67929.0,84.0,46.725824,26.690,0.612106,15.79,15.98,call,put,...,9.62,-0.394721,29.229,46.497207,318.0,19160.0,SPY,20210416,392.3,25.177
429,386.0,6602.0,11.0,46.329128,26.358,0.599909,15.14,15.19,call,put,...,9.91,-0.406036,28.912,46.154843,32.0,2315.0,SPY,20210416,392.3,25.177
430,387.0,6744.0,20.0,45.972996,26.059,0.587269,14.44,14.50,call,put,...,10.22,-0.417666,28.605,45.848527,133.0,3737.0,SPY,20210416,392.3,25.177
431,388.0,6470.0,48.0,45.662571,25.755,0.574287,13.76,13.81,call,put,...,10.55,-0.429599,28.308,45.580175,30.0,2071.0,SPY,20210416,392.3,25.177
432,389.0,1975.0,13.0,45.399371,25.458,0.560944,13.08,13.15,call,put,...,10.88,-0.441813,28.009,45.351849,37.0,2117.0,SPY,20210416,392.3,25.177
433,390.0,14874.0,104.0,45.185776,25.177,0.547241,12.43,12.50,call,put,...,11.24,-0.454322,27.729,45.164983,345.0,11867.0,SPY,20210416,392.3,25.177
434,391.0,3811.0,30.0,45.024722,24.890,0.533212,11.79,11.86,call,put,...,11.60,-0.467095,27.433,45.021832,25.0,1581.0,SPY,20210416,392.3,25.177
435,392.0,3808.0,74.0,44.918784,24.605,0.518855,11.17,11.23,call,put,...,11.97,-0.480138,27.154,44.924293,122.0,823.0,SPY,20210416,392.3,25.177
